In [1]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [158]:
import os, pandas as pd
os.chdir('/content/drive/MyDrive/jiwon')

weather = pd.read_csv('해운대 날씨3.csv')
customer = pd.read_csv('해운대 입장객수3.csv')

In [159]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers import Bidirectional
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import datetime
import time
import inspect

# 날짜를 date type으로 변경 후, 나머지는 numeric type으로 변경
weather['날짜'] = pd.to_datetime(weather['날짜'], infer_datetime_format=True)
weather.iloc[:,1:] = weather.iloc[:,1:].apply(pd.to_numeric)
# 모든행,첫번째열(0번부터니까 두번째라는말) apply=모든값에 적용하겠다
customer['방문일'] = pd.to_datetime(customer['방문일'], infer_datetime_format=True)
customer['방문객수'] = customer['방문객수'].str.replace(",","")
customer.iloc[:,1:] = customer.iloc[:,1:].apply(pd.to_numeric)


In [160]:
# 데이터 합치기
total_data = pd.merge(weather, customer, left_on='날짜', right_on="방문일", how='inner')
total_data = total_data[['강수_관측값', "기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기", "방문객수"]]

In [161]:
# 결측치 평균, 0 채우기
total_data.loc[total_data['평균수온'] != total_data['평균수온'], '평균수온'] = total_data['평균수온'].mean()
total_data.loc[total_data['평균풍속'] != total_data['평균풍속'], '평균풍속'] = total_data['평균풍속'].mean()
total_data.loc[total_data['평균기압'] != total_data['평균기압'], '평균기압'] = total_data['평균기압'].mean()
total_data.loc[total_data['평균최대파고'] != total_data['평균최대파고'], '평균최대파고'] = total_data['평균최대파고'].mean()
total_data.loc[total_data['평균파주기'] != total_data['평균파주기'], '평균파주기'] = total_data['평균파주기'].mean()
total_data.loc[total_data['방문객수'] != total_data['방문객수'], '방문객수'] = total_data['방문객수'].mean()


In [162]:
# input data
weather_input = total_data[['강수_관측값',"기온", "습도", "체감온도", "평균수온"]]
weather_output = total_data[['강수_관측값','기온']]

In [163]:
# train/test split
train_x, test_x, train_y, test_y = train_test_split(weather_input, weather_output, test_size=0.2, random_state=42)

In [164]:
# minmax scaler
min_scaler = MinMaxScaler()
train_x_scaled = min_scaler.fit_transform(train_x)
test_x_scaled = min_scaler.transform(test_x)

In [165]:
train_x = train_x_scaled.astype(float)
test_x = test_x_scaled.astype(float)
train_y = train_y.astype(float)
test_y = test_y.astype(float)

In [166]:
train_x.shape, test_x.shape

((1175, 5), (294, 5))

In [154]:
train_x = train_x_scaled.reshape(1175, 5, 1)
test_x = test_x_scaled.reshape(294, 5, 1)

In [169]:
from keras.layers.recurrent import SimpleRNN, GRU
# 2.모델구성 / 날씨
model=Sequential()
model.add(LSTM(units=256,input_shape=(5,1),return_sequences=True))
model.add(Bidirectional(LSTM(512)))
model.add(Dense(256,activation='swish'))
model.add(Dense(128,activation='swish'))
model.add(Dense(64,activation='swish'))
model.add(Dense(32,activation='swish'))
model.add(Dense(16,activation='swish'))
model.add(Dense(2))
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 5, 256)            264192    
                                                                 
 bidirectional_10 (Bidirecti  (None, 1024)             3149824   
 onal)                                                           
                                                                 
 dense_60 (Dense)            (None, 256)               262400    
                                                                 
 dense_61 (Dense)            (None, 128)               32896     
                                                                 
 dense_62 (Dense)            (None, 64)                8256      
                                                                 
 dense_63 (Dense)            (None, 32)                2080      
                                                     

In [170]:
# 3. 컴파일 훈련
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, mode='min', verbose=1,restore_best_weights=True)
model.compile(loss='mean_squared_error', optimizer='adam')
hist = model.fit(train_x, train_y, epochs=1000, batch_size=1024, 
                validation_split=0.2,
                callbacks = [earlyStopping],
                verbose=2)

Epoch 1/1000
1/1 - 9s - loss: 645.3351 - val_loss: 551.7858 - 9s/epoch - 9s/step
Epoch 2/1000
1/1 - 2s - loss: 644.6813 - val_loss: 550.8207 - 2s/epoch - 2s/step
Epoch 3/1000
1/1 - 2s - loss: 643.7212 - val_loss: 549.0239 - 2s/epoch - 2s/step
Epoch 4/1000
1/1 - 2s - loss: 641.9353 - val_loss: 545.5453 - 2s/epoch - 2s/step
Epoch 5/1000
1/1 - 2s - loss: 638.4830 - val_loss: 538.9469 - 2s/epoch - 2s/step
Epoch 6/1000
1/1 - 2s - loss: 631.9642 - val_loss: 527.2274 - 2s/epoch - 2s/step
Epoch 7/1000
1/1 - 2s - loss: 620.5291 - val_loss: 508.0077 - 2s/epoch - 2s/step
Epoch 8/1000
1/1 - 2s - loss: 602.1225 - val_loss: 466.7501 - 2s/epoch - 2s/step
Epoch 9/1000
1/1 - 2s - loss: 562.4752 - val_loss: 387.8346 - 2s/epoch - 2s/step
Epoch 10/1000
1/1 - 2s - loss: 485.2086 - val_loss: 285.9104 - 2s/epoch - 2s/step
Epoch 11/1000
1/1 - 2s - loss: 384.2484 - val_loss: 211.3547 - 2s/epoch - 2s/step
Epoch 12/1000
1/1 - 2s - loss: 309.0547 - val_loss: 215.6819 - 2s/epoch - 2s/step
Epoch 13/1000
1/1 - 2s - 

In [171]:
# 4. 평가, 예측
loss=model.evaluate(test_x,test_y)
pred_y=model.predict(test_x)

print('loss: ',loss)
print('7월 31일의 예상 강수량, 기온 : ', pred_y[-1:])

10/10 [==============================] - 0s 37ms/step - loss: 2.1299
loss:  2.1299238204956055
7월 31일의 예상 강수량, 기온 :  [[ 0.27191058 22.037153  ]]


*swish / relu 
loss:  249.6083984375
예상 강수량, 기온 :  [[ 6.5666695 21.711378 ]]
*relu
loss:  212.92079162597656
예상 강수량, 기온 :  [[11.745576 16.83425 ]]
*swish
loss:  247.30873107910156
예상 강수량, 기온 :  [[ 7.923085 23.06996 ]]
*tahn
loss:  258.86114501953125
예상 강수량, 기온 :  [[ 6.8169184 26.499884 ]]

*SimpleRNN 
loss:  11.540485382080078
예상 강수량, 기온 :  [[-0.5541291 26.209843 ]]
*LSTM
loss:  4.436618328094482
예상 강수량, 기온 :  [[ 0.7016405 24.860674 ]]
*GRU
loss:  227.4101104736328
예상 강수량, 기온 :  [[ 6.748581 22.655369]]

loss:  0.03031637892127037 (1,5)
7월 31일의 예상 강수량, 기온 :  [[ 0.07060577 22.014408  ]]

loss:  2.1299238204956055 (5,1)
7월 31일의 예상 강수량, 기온 :  [[ 0.27191058 22.037153  ]]